In [ ]:
import cv2
import numpy as np

# Diccionario de colores básicos
color_dict = {
    "Rojo": (255, 0, 0), "Verde": (0, 255, 0), "Azul": (0, 0, 255),
    "Amarillo": (255, 255, 0), "Cian": (0, 255, 255), "Magenta": (255, 0, 255),
    "Negro": (0, 0, 0), "Blanco": (255, 255, 255), "Gris": (128, 128, 128),
    "Naranja": (255, 165, 0), "Rosa": (255, 192, 203), "Marron": (139, 69, 19),
    "Violeta": (148, 0, 211),
}

def get_color_name(b, g, r):
    """Devuelve el nombre del color más cercano"""
    min_dist = float("inf")
    closest_color = "Desconocido"
    for name, (cr, cg, cb) in color_dict.items():
        dist = np.sqrt((r - cr) ** 2 + (g - cg) ** 2 + (b - cb) ** 2)
        if dist < min_dist:
            min_dist = dist
            closest_color = name
    return closest_color

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (640, 480))

    # Aplicar desenfoque para reducir ruido
    blurred = cv2.GaussianBlur(frame, (5, 5), 0)

    # Convertir a escala de grises
    gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)
    
    # Prueba: Mostrar imagen en escala de grises
    cv2.imshow("Escala de Grises", gray)

    # Aplicar umbral adaptativo
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY_INV, 21, 10)
    # Prueba: Mostrar imagen binaria
    cv2.imshow("Umbral", thresh)

    # Encontrar contornos
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    print(f"Contornos detectados: {len(contours)}")  # Depuración

    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 1500:
            continue  # Ignorar pequeños objetos

        # Aproximar el contorno
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # Dibujar contorno
        cv2.drawContours(frame, [approx], 0, (0, 0, 0), 2)

        # Obtener la caja delimitadora
        x, y, w, h = cv2.boundingRect(approx)

        # Obtener color promedio en el centro de la figura
        cx, cy = x + w // 2, y + h // 2
        roi = frame[cy - 5: cy + 5, cx - 5: cx + 5]
        avg_color = np.mean(roi, axis=(0, 1)) if roi.size > 0 else (0, 0, 0)
        b, g, r = avg_color.astype(int)
        color_name = get_color_name(b, g, r)

        # Determinar la forma
        shape_name = "Desconocido"
        if len(approx) == 3:
            shape_name = "Triangulo"
        elif len(approx) == 4:
            shape_name = "Cuadrado"
        elif len(approx) == 5:
            shape_name = "Pentagono"
        elif len(approx) == 6:
            shape_name = "Hexagono"
        else:
            shape_name = "Circulo"

        # Texto con forma y color
        text = f"{shape_name}, {color_name}"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.7
        thickness = 2
        text_size, _ = cv2.getTextSize(text, font, font_scale, thickness)

        # Calcular la posición del texto
        text_x = x + (w - text_size[0]) // 2
        text_y = y - 10 if y > 20 else y + h + 30

        # Dibujar fondo blanco detrás del texto
        cv2.rectangle(frame, (text_x - 5, text_y - text_size[1] - 5),
                      (text_x + text_size[0] + 5, text_y + 5), (255, 255, 255), -1)

        # Dibujar texto
        cv2.putText(frame, text, (text_x, text_y), font, font_scale, (0, 0, 0), thickness)

    # Mostrar la imagen procesada
    cv2.imshow("Detección de Formas y Colores", frame)

    # Salir con 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Contornos detectados: 201
Contornos detectados: 208
Contornos detectados: 196
Contornos detectados: 200
Contornos detectados: 193
Contornos detectados: 179
Contornos detectados: 180
Contornos detectados: 177
Contornos detectados: 172
Contornos detectados: 177
Contornos detectados: 192
Contornos detectados: 185
Contornos detectados: 182
Contornos detectados: 185
Contornos detectados: 187
Contornos detectados: 187
Contornos detectados: 197
Contornos detectados: 192
Contornos detectados: 190
Contornos detectados: 190
Contornos detectados: 198
Contornos detectados: 197
Contornos detectados: 190
Contornos detectados: 202
Contornos detectados: 191
Contornos detectados: 188
Contornos detectados: 185
Contornos detectados: 189
Contornos detectados: 195
Contornos detectados: 193
Contornos detectados: 189
Contornos detectados: 186
Contornos detectados: 189
Contornos detectados: 198
Contornos detectados: 176
Contornos detectados: 186
Contornos detectados: 190
Contornos detectados: 184
Contornos de